In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
%matplotlib inline
from pycaret.classification import *

In [ ]:
# Three sensor data string
sensorType = ['data_pdmp', 'data_pin', 'data_po']

def read_data(sensorType, max_lengths):
    '''
        Read data.
        return data(pandas dataframe)
    '''
    data = []
    # Find the data files with different types(sensors).
    for i in range(len(sensorType)):
        targetPattern = "training_data/"+sensorType[i]+"*.csv"
        fileList = glob.glob(targetPattern)
        # Read data
        column_names = [j for j in range(0, max_lengths[i])]
        df = pd.concat((pd.read_csv(file, header=None, delimiter=',', names=column_names) for file in fileList), ignore_index=True)
        #data = pd.read_csv("../datasets/features/dataset.csv", header=None, delimiter=',', names=column_names)
        data.append(df)
    return data
    

def calc_max_length(sensorType):
    '''
        Calculate max length of time series with different data.
        return max_column_count(List[int])
    '''
    max_column_count = []
    for i in range(len(sensorType)):
        targetPattern = "training_data/"+sensorType[i]+"*.csv"
        fileList = glob.glob(targetPattern)
        largest_column_count = 0
        # Read data
        for csv_file in fileList:
            with open(csv_file, 'r') as temp_f:
                lines = temp_f.readlines()
                for l in lines:
                    column_count = len(l.split(',')) + 1
                    largest_column_count = column_count if largest_column_count < column_count else largest_column_count
            temp_f.close()
        max_column_count.append(largest_column_count)
    return max_column_count

# 取得最長的資料長度 讓pandas讀取時保持一致的Columns數量 方便後續的資料處理
max_lengths = calc_max_length(sensorType)
data = read_data(sensorType, max_lengths)
# 合併資料並刪除重複的Label
df = pd.concat((data[0], data[1].drop([0], axis=1), data[2].drop([0], axis=1)), axis='columns')
df.columns = [str(i) for i in range(df.shape[1])]

In [ ]:
# 讓PyCaret自動選擇前處理方法
s = setup(df, target = '0')

In [ ]:
# 以不同Model的預設參數進行訓練  預設為10-fold Cross validation
best = compare_models()